In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [2]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(504818, 124)

In [3]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [4]:
model_data = data.sample(frac=.1)
model_data.shape

(50482, 124)

In [5]:
X_train = model_data.drop(['is_churn'], axis=1);
y_train = model_data['is_churn'];

In [6]:
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
Unnamed: 0.1.1                  True
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
m

In [7]:
print(np.all(np.isfinite(X_train)))
print(np.any(np.isnan(X_train)))

False
True


In [8]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train, scoring="neg_log_loss", train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [21]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_xgb = {
    'n_estimators':[100, 200, 300, 400, 500],
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2),
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
    }
#colsample_bytree=0.636221066412, gamma=0.859598446221, max_depth=7, min_child_weight=3, reg_alpha=3.59003832809, reg_lambda=0.312604869001, subsample=0.586018102037, score=-0.07374870982572349, total= 
param_xgb = {
    'n_estimators':[100, 200, 300, 400, 500],
    'max_depth':range(3,10,2),
    'min_child_weight':[3, 6, 8],
    'gamma':[.5,0.859598446221, 1],
    'subsample':[0.5, 0.586018102037, .65],
    'colsample_bytree' : [0.6, 0.636221066412, 0.8],
    'reg_alpha':[0.1, 1, 3.59003832809, 10],
    'reg_lambda':[0.312604869001]
    }
param_lgbm = {
    'learning_rate': [0.005, .05, .1, .5],
    'n_estimators': [8,16,24, 50, 100, 150],
    'num_leaves': [6,8,12,16, 32, 64],
    'max_depth': [-1, 7, 10],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.6, 0.636221066412, 0.8],
    'subsample' : [0.5, 0.7,1],
    'reg_alpha' : [0, 0.5, 1, 1.2, 1.4],
    'reg_lambda' : [0, 0.5, 1 ,1.2, 1.4],
}

param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [22]:
model_list = {LogisticRegression():param_logit, KNeighborsClassifier():param_knn,
             SVC():param_svc, RandomForestClassifier():param_rfc};
model_list = {LogisticRegression():param_logit}
model_list = {XGBClassifier(n_jobs=3, verbose=True): param_xgb}
#model_list = {LGBMClassifier(nthread=3): param_lgbm}

In [ ]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)

learning curve for	XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=3, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, verbose=True)


In [18]:
# Tune the parameters in a randomized fashion and control the distributions for colsample_bytree and subsample
from xgboost_tuner.tuner import tune_xgb_params

best_params, history = tune_xgb_params(
    cv_folds=3,
    label=y_train,
    metric_sklearn='neg_log_loss',
    metric_xgb='logloss',
    n_jobs=4,
    objective='binary:logistic',
    random_state=2017,
    strategy='randomized',
    train=X_train,
    colsample_bytree_loc=0.5,
    colsample_bytree_scale=0.2,
    subsample_loc=0.5,
    subsample_scale=0.2
)

[0]	train-logloss:0.606908+8.80946e-05	test-logloss:0.607269+0.000512625
[1]	train-logloss:0.536115+0.000152154	test-logloss:0.536634+0.000726114
[2]	train-logloss:0.477156+0.000140108	test-logloss:0.477912+0.000909318
[3]	train-logloss:0.427216+0.000184391	test-logloss:0.428228+0.00130051
[4]	train-logloss:0.384421+0.000113685	test-logloss:0.3856+0.00125145
[5]	train-logloss:0.347684+0.000100364	test-logloss:0.349005+0.00142289
[6]	train-logloss:0.316096+0.000107252	test-logloss:0.3177+0.00161207
[7]	train-logloss:0.28836+0.000367854	test-logloss:0.290132+0.00159999
[8]	train-logloss:0.264074+0.000508853	test-logloss:0.266011+0.00160042
[9]	train-logloss:0.242843+0.000665438	test-logloss:0.244978+0.00170367
[10]	train-logloss:0.224113+0.000856367	test-logloss:0.226475+0.00172524
[11]	train-logloss:0.207498+0.000935356	test-logloss:0.210076+0.00173122
[12]	train-logloss:0.192808+0.00103361	test-logloss:0.195566+0.00170456
[13]	train-logloss:0.179753+0.00108268	test-logloss:0.182752+0.0

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   45.9s


[CV]  colsample_bytree=0.532799688421, gamma=0.615008304643, max_depth=3, min_child_weight=1, reg_alpha=3.47357399604, reg_lambda=2.08369567338, subsample=0.524762128398, score=-0.08066524113512104, total=  11.5s
[CV]  colsample_bytree=0.532799688421, gamma=0.615008304643, max_depth=3, min_child_weight=1, reg_alpha=3.47357399604, reg_lambda=2.08369567338, subsample=0.524762128398, score=-0.07815195019962402, total=  11.4s
[CV] colsample_bytree=0.57286519454, gamma=0.66662566959, max_depth=2, min_child_weight=7, reg_alpha=1.29378189347, reg_lambda=2.74494343424, subsample=0.648129466185 
[CV] colsample_bytree=0.57286519454, gamma=0.66662566959, max_depth=2, min_child_weight=7, reg_alpha=1.29378189347, reg_lambda=2.74494343424, subsample=0.648129466185 
[CV]  colsample_bytree=0.519535708773, gamma=0.722971048235, max_depth=5, min_child_weight=10, reg_alpha=0.58628494269, reg_lambda=1.07591486694, subsample=0.637259533451, score=-0.07991704402544778, total=  19.1s
[CV] colsample_bytree=0.

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.0min


[CV]  colsample_bytree=0.57286519454, gamma=0.66662566959, max_depth=2, min_child_weight=7, reg_alpha=1.29378189347, reg_lambda=2.74494343424, subsample=0.648129466185, score=-0.08894665405982384, total=   9.6s
[CV] colsample_bytree=0.581646850967, gamma=0.0345517650555, max_depth=2, min_child_weight=1, reg_alpha=3.33961420141, reg_lambda=1.35883511621, subsample=0.649440023705 
[CV]  colsample_bytree=0.581646850967, gamma=0.0345517650555, max_depth=2, min_child_weight=1, reg_alpha=3.33961420141, reg_lambda=1.35883511621, subsample=0.649440023705, score=-0.08705414557139475, total=  10.1s
[CV] colsample_bytree=0.581646850967, gamma=0.0345517650555, max_depth=2, min_child_weight=1, reg_alpha=3.33961420141, reg_lambda=1.35883511621, subsample=0.649440023705 
[CV]  colsample_bytree=0.581646850967, gamma=0.0345517650555, max_depth=2, min_child_weight=1, reg_alpha=3.33961420141, reg_lambda=1.35883511621, subsample=0.649440023705, score=-0.08348254670127815, total=  10.5s
[CV] colsample_bytr

...

[CV]  colsample_bytree=0.553420024376, gamma=0.199888144056, max_depth=5, min_child_weight=5, reg_alpha=2.76626768928, reg_lambda=3.71340488776, subsample=0.574268053036, score=-0.07736842857512659, total=  16.6s
[CV] colsample_bytree=0.648160917532, gamma=0.804606528344, max_depth=6, min_child_weight=5, reg_alpha=2.97027994972, reg_lambda=5.37620224441, subsample=0.654757872058 
[CV]  colsample_bytree=0.553420024376, gamma=0.199888144056, max_depth=5, min_child_weight=5, reg_alpha=2.76626768928, reg_lambda=3.71340488776, subsample=0.574268053036, score=-0.07456664530217334, total=  16.6s
[CV] colsample_bytree=0.648160917532, gamma=0.804606528344, max_depth=6, min_child_weight=5, reg_alpha=2.97027994972, reg_lambda=5.37620224441, subsample=0.654757872058 
[CV]  colsample_bytree=0.553420024376, gamma=0.199888144056, max_depth=5, min_child_weight=5, reg_alpha=2.76626768928, reg_lambda=3.71340488776, subsample=0.574268053036, score=-0.0799913062147108, total=  16.5s
[CV] colsample_bytree=

[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  5.5min


[CV]  colsample_bytree=0.533217188916, gamma=0.656059395972, max_depth=5, min_child_weight=8, reg_alpha=2.86487091931, reg_lambda=5.14474649169, subsample=0.563690736733, score=-0.08123982018657365, total=  21.7s
[CV] colsample_bytree=0.636221066412, gamma=0.859598446221, max_depth=7, min_child_weight=3, reg_alpha=3.59003832809, reg_lambda=0.312604869001, subsample=0.586018102037 
[CV]  colsample_bytree=0.570347063478, gamma=0.455802928782, max_depth=3, min_child_weight=7, reg_alpha=7.15119370128, reg_lambda=9.30142764981, subsample=0.630451463197, score=-0.08220229170728523, total=  14.0s
[CV] colsample_bytree=0.636221066412, gamma=0.859598446221, max_depth=7, min_child_weight=3, reg_alpha=3.59003832809, reg_lambda=0.312604869001, subsample=0.586018102037 
[CV]  colsample_bytree=0.570347063478, gamma=0.455802928782, max_depth=3, min_child_weight=7, reg_alpha=7.15119370128, reg_lambda=9.30142764981, subsample=0.630451463197, score=-0.07926782624110118, total=  13.7s
[CV] colsample_bytr

[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:  6.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:  6.1min finished


[0]	train-logloss:0.684177+6.08075e-05	test-logloss:0.684203+9.0116e-05
[1]	train-logloss:0.675335+6.0802e-05	test-logloss:0.675378+0.000119114
[2]	train-logloss:0.666819+0.0001666	test-logloss:0.666885+0.000128645
[3]	train-logloss:0.658495+0.000281771	test-logloss:0.658589+0.000340807
[4]	train-logloss:0.65014+0.000281259	test-logloss:0.650249+0.000329243
[5]	train-logloss:0.641966+0.000285244	test-logloss:0.642092+0.000348785
[6]	train-logloss:0.634025+0.00028145	test-logloss:0.634173+0.00039073
[7]	train-logloss:0.626141+0.000262095	test-logloss:0.626301+0.000385609
[8]	train-logloss:0.61855+0.000426214	test-logloss:0.618734+0.000619186
[9]	train-logloss:0.61109+0.000236683	test-logloss:0.611295+0.000514966
[10]	train-logloss:0.60364+0.000213084	test-logloss:0.603858+0.000509413
[11]	train-logloss:0.596354+0.00017937	test-logloss:0.596584+0.00050843
[12]	train-logloss:0.58916+0.000160824	test-logloss:0.589403+0.000498175
[13]	train-logloss:0.582103+0.00012598	test-logloss:0.582364+